In [15]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.saving import register_keras_serializable

In [16]:
california_housing = fetch_california_housing()

X_train0, X_test, Y_train0, Y_test = train_test_split(california_housing["data"], california_housing["target"])
X_train1, X_validation, Y_train1, Y_validation = train_test_split(X_train0, Y_train0)

sc = StandardScaler()
X_train_s = sc.fit_transform(X_train1)
X_validation_s = sc.fit_transform(X_validation)
X_test_s = sc.transform(X_test)


X_train_s_1, X_train_s_2 = X_train_s[:, :6], X_train_s[:, :-4]
X_validation_s_1, X_validation_s_2 = X_validation_s[:, :6], X_validation_s[:, :-4]
X_test_s_1, X_test_s_2 = X_test_s[:, :6], X_test_s[:, :-4]


@register_keras_serializable()
class WideAndDeepANN(keras.Model):
    def __init__(self, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden_layer_1 = keras.layers.Dense(50, activation=activation)
        self.hidden_layer_2 = keras.layers.Dense(10, activation=activation)
        self.final_output = keras.layers.Dense(1)
        self.helper_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_1, input_2 = inputs
        h1_out = self.hidden_layer_1(input_1)
        h2_out = self.hidden_layer_2(h1_out)
        concat_out = keras.layers.concatenate([input_2, h2_out])
        final_out = self.final_output(concat_out)
        helper_out = self.helper_output(h2_out)
        return final_out, helper_out


model_f = WideAndDeepANN()

model_checkpoint_callback = keras.callbacks.ModelCheckpoint("model_cb_regression_housing.keras", save_best_only=True)
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
tb_callback = keras.callbacks.TensorBoard("tb_log3")

model_f.compile(loss=["mse", "mse"],
                loss_weights=[0.8, 0.2],
                optimizer="sgd",
                metrics=[["mean_absolute_error"], ["mean_absolute_error"]])

model_f.fit((X_train_s_1, X_train_s_2), (Y_train1, Y_train1), epochs=2,
            validation_data=((X_validation_s_1, X_validation_s_2), (Y_validation, Y_validation)),
            callbacks=[tb_callback])
model_f.save("housing_reg_model.keras")
model_f_reg = keras.models.load_model("housing_reg_model.keras")

class Mycallback(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        print("")

    def on_epoch_end(self, epoch, logs):
        print(logs["var_loss"])


mycb = Mycallback

def ann_model(number_of_hidden_layers=1,
              number_of_neurons=50,
              lr=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=[8]))
    for hidden_layer in range(number_of_hidden_layers):
        model.add(keras.layers.Dense(number_of_neurons, activation="selu"))
    model.add(keras.layers.Dense(1))
    sgd = keras.optimizers.SGD(learning_rate=lr)
    model.compile(loss="mse", optimizer=sgd)
    return model

Epoch 1/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4865 - mean_absolute_error: 0.8262 - mean_absolute_error_1: 1.0440 - mse_loss: 2.0085 - val_loss: 0.5971 - val_mean_absolute_error: 0.5465 - val_mean_absolute_error_1: 0.5925 - val_mse_loss: 0.6442
Epoch 2/2
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5809 - mean_absolute_error: 0.5519 - mean_absolute_error_1: 0.5801 - mse_loss: 0.6122 - val_loss: 0.5414 - val_mean_absolute_error: 0.5259 - val_mean_absolute_error_1: 0.5537 - val_mse_loss: 0.5737


In [ ]:
# keras_sk_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=ann_model)

In [18]:
keras_sk_reg = keras.wrappers.SKLearnRegressor(model=ann_model)